In [1]:
!pip install ffmpeg 

In [15]:
!pip install ffmpeg-python

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached future-0.18.2-py3-none-any.whl
  Attempting uninstall: future
    Found existing installation: future 0.18.2
    Uninstalling future-0.18.2:
      Successfully uninstalled future-0.18.2
  Attempting uninstall: ffmpeg-python
    Found existing installation: ffmpeg-python 0.2.0
    Uninstalling ffmpeg-python-0.2.0:
      Successfully uninstalled ffmpeg-python-0.2.0


In [1]:
import ffmpeg

In [2]:
video_path = "yt_vids/OYN8mZ5nCQY.mp4"

In [3]:
probe = ffmpeg.probe(video_path)

vid_probe = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)

if vid_probe is None:

    print('[ERROR]: No video stream found', file=sys.stderr)
    
else:
    
    width = int(vid_probe['width'])
    height = int(vid_probe['height'])
    codec = str(vid_probe['codec_name'])
    duration = float(vid_probe['duration'])
    frame_rate = str(vid_probe['r_frame_rate'])
    
print("width: {}, height: {}, codec: {}, duration: {}, frame rate: {}".format(width, height, codec, duration, frame_rate))
    

width: 1280, height: 720, codec: h264, duration: 105.36, frame rate: 25/1


In [5]:
stream = (
    ffmpeg
    .input(
        video_path,
        ss = 2.0, # where to seek the video, in seconds
        t = 3.0 # duration of clip, in seconds
    )
)

out_path = 'yt_vids/test_clip.mp4'

ffmpeg.output(stream, stream.audio, out_path, format='mp4', vcodec='h264', acodec='aac', r=frame_rate).global_args('-loglevel', 'quiet').overwrite_output().run()

(None, None)

In [11]:
from IPython.display import Video

Video(out_path)

In [18]:
import random
import os

clip_dir = "random_clips"

if not os.path.exists(clip_dir):
    
    os.mkdir(clip_dir)
        
def get_random_clip(video):
    
    clip_duration = 1.0

    max_time = duration - clip_duration
    
    rnd_start = random.uniform(0.0, max_time)
    
    stream = (
        ffmpeg
        .input(
            video,        
            ss = rnd_start, # where to seek the video, in seconds
            t = max_time # duration of clip, in seconds
        )
    )
    
    return stream
    
clip_amount = 5

for i in range(clip_amount):
    
    stream = get_random_clip(video_path)
    
    out_path = os.path.join(clip_dir, "clip_{}.mp4".format(i+1))

    ffmpeg.output(stream, stream.audio, out_path, format='mp4', vcodec='h264', acodec='aac', r=frame_rate).global_args('-loglevel', 'quiet').overwrite_output().run()
    
    # clips.append(rnd_clip)

In [23]:
clip_paths = []
    
for path, subdirs, files in os.walk(clip_dir):

    for name in files:

        filename, file_extension = os.path.splitext(name)

        if file_extension == '.mp4':
            
            clip_paths.append(filename)

clips = []
for path in clip_paths:
    
    stream = (
        ffmpeg
        .input(
            path        
        )
    )
    clips.append(stream)

all_clips = ffmpeg.concat(*clips)

out_dir = "ffmpeg_out"

if not os.path.exists(out_dir):
    
    os.mkdir(out_dir)

out_path = os.path.join(out_dir, "random_comp.mp4")

ffmpeg.output(all_clips, all_clips.audio, out_path, format='mp4', vcodec='h264', acodec='aac', r=frame_rate).global_args('-loglevel', 'quiet').overwrite_output().run()



ValueError: Encountered concat(n=5) <d70838d97fd8> with multiple outgoing edges with same upstream label None; a `split` filter is probably required